# Using memmap
Most of the times you will have to deal with data sets that are to big to fit in RAM. The solution to manipulating those is to use numpy.memmmap. That function gives you a pointer to memory in the hard disk where your data lives but is treated by numpy as a normal array.
The thing with memmap arrays is that if you pass them to functions that do array manipulation then numpy will try to load the whole array in RAM. That will crash your computer. So manipulating such arrays needs to be done in parts (think loops). The trick here is to do this as efficiently as possible.

## Random matrix and its covariance. Difficulty 2/5
Let's start by setting up the problem. You will understand what the covariance function of numpy does by recreating it and runnign it on small arrays. This will give you an idea of how much slower the naive (loop based) approach is to the highly optimized numpy (vectorized) approach.

1. Generate a random elements matrix with 1000 rows (the samples) and 100 columns (the dimensions).

2. Calculate the covariance of this matrix using numpy.cov

3. Time this

4. Make some pairs of the matrix samples partially correlated<sup>*</sup> with each other. Create 200 such pairs (they should be randomly placed in the matrix but the relative position of the two samples in a pair can be either random or fixed). 

5. Report some covariances of the two matrices that show that on the pairs of samples you have chossen to increase the covariance that has happened.

Notes:

1. <sup>*</sup> To get perfect covariance then two samples would have to be identical (i.e. just copy one sample to the place of another). To get partial covariance one sample would have to be a little bit different to the other. That can be achieved by multiplying a copied sample's elements with random numbers that are close to one.

2. Do not use loops for any of the above. Everything should be achieved with numpy.

In [ ]:
import numpy as np
import time

matrix_dimensions = (1000,100)
correlated_pairs_amount = 200

matrix = np.random.random(matrix_dimensions)

t = time.clock()
first_cov = np.cov(matrix)
print('Time = ' + str(time.clock() - t) + 'sec')

correlated_pairs_distances = np.random.randint(1, 100, correlated_pairs_amount)
correlated_pairs_indices_first = np.random.randint(101, matrix_dimensions[0], correlated_pairs_amount)
correlated_pairs_indices_second = correlated_pairs_indices_first - correlated_pairs_distances

print('Average covariance of the pairs = ' + \
      str(np.mean(first_cov[correlated_pairs_indices_first, correlated_pairs_indices_second])))
print('Covariance of a diagonal element = ' + str(first_cov[0,0]))

correlation_additive = np.random.uniform(0.8, 0.9, (correlated_pairs_amount, matrix_dimensions[1]))

matrix_cov = np.copy(matrix)
matrix_cov[correlated_pairs_indices_first] = np.copy(matrix[correlated_pairs_indices_second]) * correlation_additive
second_cov = np.cov(matrix_cov)

print('Average covariance of the pairs = ' + \
      str(np.mean(second_cov[correlated_pairs_indices_first, correlated_pairs_indices_second])))
print('Covariance of a diagonal element = ' + str(second_cov[0,0]))

## Calculate the covariance of the same matrices using a multiple loop. Difficulty 3/5

Now try and calulate the covariance by the most naive, loop based way possible.

Have a look [here](http://stattrek.com/matrix-algebra/covariance-matrix.aspx) for a quick review of covariance

Do things one element at a time and use as little numpy as possible (use it to calculate the row means)

Check that the results of np.cov and your results are the same (print the mean of the square difs)

Time how long that takes

In [ ]:
def covariance(matrix1, matrix2):
    result = np.empty((matrix1.shape[0], matrix2.shape[0]))
    
    for i in range(matrix1.shape[0]):
        mean1 = np.mean(matrix1[i])
        for j in range(matrix2.shape[0]):
            mean2 = np.mean(matrix2[j])
            temp_sum = 0
            for k in range(matrix1.shape[1]):
                temp_sum += (matrix1[i, k] - mean1) * (matrix2[j, k] - mean2)
            result[i, j] = temp_sum / matrix1.shape[1]
    
    return result

t = time.clock()
loop_first_cov = covariance(matrix, matrix)
print('Time = ' + str(time.clock() - t) + 'sec')
    
print('First_Diff^2 = ' + str(np.mean(np.power(first_cov - loop_first_cov, 2))))

t = time.clock()
loop_second_cov = covariance(matrix_cov, matrix_cov)
print('Time = ' + str(time.clock() - t) + 'sec')
    
print('Diff^2 = ' + str(np.mean(np.power(second_cov - loop_second_cov, 2))))

## Generate a large matrix (not fitting into RAM). Difficulty 1/5

Use numpy.memmap to create a matrix whose elements are kept in the hard disk and not in RAM and that is so large that keeping it into RAM would be impossible.
Make the matrix 10<sup>5</sup> x 10<sup>5</sup> elements. 

Fill this with random numbers.

How much memory will that take? What else do you need to know to calculate the memory it will occupy other than its size?


In [ ]:
from os import path as path
folder = r'E:\Data\temp_for_pystarters'

In [ ]:
matrix_file = path.join(folder, 'big_matrix.dat')
matrix_dimensions = (10**5, 10**5)

big_matrix_mm = np.memmap(matrix_file, np.float32, mode='w+', shape=matrix_dimensions)
big_matrix_mm[:] = np.random.random(matrix_dimension)
for i in range(big_matrix_mm.shape[0]):
    big_matrix_mm[i, :] = np.random.random(big_matrix_mm.shape[1])
print('Done')

## Calculate the covariance of this matrix. Difficulty 5/5 (this is hard, do your best)
Calulate the covariance of this matrix using
1. The np.cov straight on the memmaped matrix. What happens?
2. An as fast as possible loop (mix numpy vectorization and loops as you see fit to get the fastest possible result). To be as efficient as possible you might need to know how much RAM your system has so as to load as much as possible into memory thus minimizing your number of loop itterations. Use the psutil.virtual_memory() function (you might have to install the psutil library).

Time how long your covariance function takes.

Note:

In order to develop your new covariance function that opperates on memmaped arrays use first some small memmaped matrices that you can do fast itterations of debuging on. While you are developing make sure that the covariance of any small matrix as calculated by np.cov is the same as the one you calculate. Also time both the np.cov function and your function on different size matrices to get a feel of how much worse you are doing and how that difference increases with matrix size. Once you are sure that everything works then run your function on the very large matrix.

Make sure your function has print statements that allow you to follow its progress. It is very usefull to know if your function is running at all when it is expected to take a very long time to complete.

In [ ]:

import numpy as np
import psutil
import time

debug_matrix_file = path.join(folder, 'debug_matrix.dat')
debug_matrix_dimensions = (10**4, 10**4)

debug_matrix_mm = np.memmap(debug_matrix_file, np.float32, mode='w+', shape=debug_matrix_dimensions)

for i in range(debug_matrix_mm.shape[0]):
    debug_matrix_mm[i, :] = np.random.random(debug_matrix_mm.shape[1])
print('Finished making matrix')

t = time.clock()
np_cov = np.cov(debug_matrix_mm)
print('Full numpy covariance took ' + str(time.clock() - t) + ' secs')


def calculate_number_of_elements_fitting_in_ram():
    remaining_memory = psutil.virtual_memory()[1]
    mem_to_use = 0.7 * remaining_memory

    total_elements_to_load_to_ram = int(mem_to_use / np.dtype(np.float32).itemsize)

    # Use this to simulate smaller ram than what is in the computer for debugging
    total_elements_to_load_to_ram = 1000

    elements_of_matrix_to_ram = int(total_elements_to_load_to_ram / 2)

    return elements_of_matrix_to_ram


def get_number_of_iterations(matrix, num_of_elements):
    total_rows = matrix.shape[0]
    return int(np.ceil(total_rows / num_of_elements))


def load_part_of_memmaped_matrix(matrix, num_of_elements_to_load, iteration_number):
    starting_index = iteration_number * num_of_elements_to_load
    ending_index = (iteration_number + 1) * num_of_elements_to_load
    if ending_index > matrix.shape[0]:
        ending_index = matrix.shape[0]

    return matrix[starting_index:ending_index, :]


def fill_in_covariance_mm(covariance, part_matrix1, part_matrix2, it1, it2, elements1, elements2):
    start_index1 = elements1 * it1
    end_index1 = elements1 * (it1 + 1)
    if end_index1 > covariance.shape[0]:
        end_index1 = covariance.shape[0]
    start_index2 = elements2 * it2
    end_index2 = elements2 * (it2 + 1)
    if end_index2 > covariance.shape[0]:
        end_index2 = covariance.shape[0]

    full_covariance = np.cov(part_matrix1, part_matrix2)
    cov_shape = full_covariance.shape
    
    # Putting results in the right place
    covariance[start_index1:end_index1, start_index2:end_index2] = full_covariance[int(cov_shape[0]/2):int(cov_shape[0])
                                                                                   , 0:int(cov_shape[1]/2)]
    covariance[start_index2:end_index2, start_index1:end_index1] = full_covariance[0:int(cov_shape[0]/2),
                                                                                   int(cov_shape[1]/2):int(cov_shape[1])]


def covariance_mm(matrix1, matrix2, result_file):

    covariance = np.memmap(result_file, np.float32, mode='w+', shape=(matrix1.shape[0], matrix2.shape[0]))

    elements_of_matrix_to_ram = calculate_number_of_elements_fitting_in_ram()

    for it1 in range(get_number_of_iterations(matrix1, elements_of_matrix_to_ram)):
        for it2 in range(get_number_of_iterations(matrix2, elements_of_matrix_to_ram)):
            part_matrix1 = load_part_of_memmaped_matrix(matrix1, elements_of_matrix_to_ram, it1)
            part_matrix2 = load_part_of_memmaped_matrix(matrix2, elements_of_matrix_to_ram, it2)
            fill_in_covariance_mm(covariance, part_matrix1, part_matrix2, it1, it2, elements_of_matrix_to_ram,
                                  elements_of_matrix_to_ram)
            
            # print(it1, it2)

    return covariance

t = time.clock()
result = covariance_mm(debug_matrix_mm, debug_matrix_mm, path.join(folder, 'debug.dat'))
print('Partial numpy covariance took ' + str(time.clock() - t) + ' secs')


dif = np.mean(np.power(result - np_cov,2))
print(dif)

# Brief intro to Pandas
Pandas is a package that makes it very easy to manipulate data sets with different labels. It's main data structures (DataFrame) are closer to excel spreadsheets and data bases than to numpy arrays. That allows you to book keep your complex data sets in intuitive ways and more importanlty to query your data (eg. Give me all the voltage timeseries for the trials where the animal was running faster than 3m/s).

Pandas is extremely powerfull and this is a very basic introduction to it. For a  little bit more have a look [here](https://pandas.pydata.org/pandas-docs/version/0.22.0/10min.html).

## Load some data into pandas DataFrames. Difficulty 2/5
Load the Events.csv and the Video.csv into Pandas DataFrames (create two seperate DataFrames).
The events file has the x, y position of a ball moving on a table, bouncing around walls. The time stamps of this file are the times that the calculation of the position of the ball happened. The video csv has the frame number of the camera recording the ball. The timestamps are the times that each frame was saved.

Use the sep parameter of the pandas.read_csv funtion to import only the date/time, the name, the x position and the y position for the events and only the date/time and the frame number for the video file. Use the read_csv function to also get pandas to transform the date/time strings into Timestamp.

The resulting DataFrames should have appropriately named columns and the data in each column should be of the correct type (Timestamp for the date/times, string for the name of the event, numbers (floats and ints) for the positions and the frame numbers).


## Merge the two DataFrames into a single one. Difficulty 3/5
Create a 3rd DataFrame that is the logical merge of the two DataFrames above. Keep only the 'common' rows. Use the rows that are closer in time between the events and the video frames.

If you turn the date/time into indices then turn them back to normal column.

The final DataFrame should have 5 columns, Time, Frame, Type, X, Y (with those or any other names you find appropriate).

## Plot the positions of the ball. Difficult 1/5
Plot all the 2d positions of the ball so that you can see its trajectory

## Create direction columns. Difficulty 2/5
Create a new column ('Direction') that records the angle (in degrees) of the movement of the ball in each frame.

To check your result redo the previous plot but now colour code each position with its direction.

## Create a DataFrame that has only the info of frames where there is a direction change. Difficulty 1/5
To check the result plot these points on top of the previous scatter plot (but bigger and red)